In [1]:
import numpy as np
import pandas as pd

from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split

In [2]:
df = pd.read_csv("recdemo2.csv", sep=";") #небольшой dataset
df

,id,A,B,C,D,E,F
0,1,3,4,2,4,1,2
1,2,3,4,2,4,1,2
2,3,3,2,5,5,1,5
3,4,5,5,5,5,4,5
4,5,3,3,2,4,4,3
5,6,1,5,5,5,3,5
6,7,1,2,2,2,1,3
7,8,1,1,1,1,4,4


In [3]:
df_unpivot=pd.melt(df, id_vars=['id'])
df_unpivot.head()

,id,variable,value
0,1,A,3
1,2,A,3
2,3,A,3
3,4,A,5
4,5,A,3


In [4]:
df_unpivot.dropna(inplace=True)
df_unpivot.columns=['userID', 'itemID', 'rating']

In [5]:
len(df_unpivot)

48

In [6]:
reader = Reader(rating_scale=(1, 5)) # Зададим разброс оценок
data = Dataset.load_from_df(df_unpivot[['userID', 'itemID', 'rating']], reader) #создадим объект, с которым умеет работать библиотека

In [7]:
trainset, testset = train_test_split(data, test_size=.30, random_state=999)

In [8]:
len(testset)

15

In [9]:
from surprise import KNNBaseline, SVD

In [17]:
algo = KNNBaseline(k=1)

In [27]:
predictions = algo.fit(trainset).test(testset)
df_ratings=pd.DataFrame(columns=['userID', 'itemID', 'rating1', 'real_rating'])
for i in predictions:
    df_ratings = df_ratings.append({'userID':i.uid, 'itemID': i.iid, 'rating1': i.est, 'real_rating': i.r_ui}, ignore_index=True)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


In [28]:
df_ratings

,userID,itemID,rating1,real_rating
0,4,B,4.973331,5.0
1,3,A,1.477356,3.0
2,6,A,5.000000,1.0
3,6,F,2.440966,5.0
4,4,F,5.000000,5.0
5,1,C,1.928561,2.0
6,2,D,4.071439,4.0
7,8,D,2.147411,1.0
8,4,D,4.973331,5.0
9,5,F,2.067312,3.0


In [29]:
algo1 = KNNBaseline(k=3)
predictions = algo1.fit(trainset).test(testset)
df_ratings2=pd.DataFrame(columns=['userID', 'itemID', 'rating2', 'real_rating'])
for i in predictions:
    df_ratings2 = df_ratings2.append({'userID':i.uid, 'itemID': i.iid, 'rating2': i.est, 'real_rating': i.r_ui}, ignore_index=True)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


In [30]:
df_ratings2

,userID,itemID,rating2,real_rating
0,4,B,4.366184,5.0
1,3,A,3.141060,3.0
2,6,A,4.637347,1.0
3,6,F,3.174512,5.0
4,4,F,3.590170,5.0
5,1,C,2.687239,2.0
6,2,D,3.762041,4.0
7,8,D,3.160107,1.0
8,4,D,4.955004,5.0
9,5,F,2.804747,3.0


In [31]:
df_ratings['rating2'] = df_ratings2['rating2']

In [14]:
algo2 = SVD(n_factors=3,random_state=999, verbose=False)
predictions = algo1.fit(trainset).test(testset)
df_ratings3=pd.DataFrame(columns=['userID', 'itemID', 'rating3', 'real_rating'])
for i in predictions:
    df_ratings3 = df_ratings.append({'userID':i.uid, 'itemID': i.iid, 'rating3': i.est, 'real_rating': i.r_ui}, ignore_index=True)

In [33]:
df_ratings['abs'] = abs(df_ratings['rating1'] - df_ratings['rating2'])

In [37]:
df_ratings[df_ratings['userID']==4]

,userID,itemID,rating1,real_rating,rating2,abs
0,4,B,4.973331,5.0,4.366184,0.607147
4,4,F,5.000000,5.0,3.590170,1.409830
8,4,D,4.973331,5.0,4.955004,0.018327


In [38]:
df_ratings

,userID,itemID,rating1,real_rating,rating2,abs
0,4,B,4.973331,5.0,4.366184,0.607147
1,3,A,1.477356,3.0,3.141060,1.663704
2,6,A,5.000000,1.0,4.637347,0.362653
3,6,F,2.440966,5.0,3.174512,0.733547
4,4,F,5.000000,5.0,3.590170,1.409830
5,1,C,1.928561,2.0,2.687239,0.758678
6,2,D,4.071439,4.0,3.762041,0.309399
7,8,D,2.147411,1.0,3.160107,1.012696
8,4,D,4.973331,5.0,4.955004,0.018327
9,5,F,2.067312,3.0,2.804747,0.737436
